# Review the Results of the NLP Pretrained Model

This model is very basic, but this folder shows you how to create an NLP pipeline with a pretrained model.

The .py file saved the results to a parquet that is stored in our container. We are reading it back here to review.

In [2]:
spark

StatementMeta(ba5e360d-f184-47a0-9859-76b5031b79e3, 41, 7, Finished, Available, Finished)

In [3]:
workspace_default_storage_account = "projectgstoragedfb938a3e"
workspace_default_container = "azureml-blobstore-becc8696-e562-432e-af12-8a5e3e1f9b0f"
workspace_wasbs_base_url = f"wasbs://{workspace_default_container}@{workspace_default_storage_account}.blob.core.windows.net/"

# the parquet path again
output_path = f"{workspace_wasbs_base_url}nlp_result_sample_submissions.parquet"

# Read the Parquet file back into a dataframe
df_read_back = spark.read.parquet(output_path)

# Show first 5 rows
df_read_back.show(5)

StatementMeta(ba5e360d-f184-47a0-9859-76b5031b79e3, 41, 8, Finished, Available, Finished)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|            sentence|               token|               spell|              lemmas|               stems|                 pos|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|Hi, I hope this i...|[{document, 0, 85...|[{document, 0, 23...|[{token, 0, 1, Hi...|[{token, 0, 1, Hi...|[{token, 0, 1, Hi...|[{token, 0, 1, hi...|[{pos, 0, 1, NNP,...|
|33 male, i have d...|[{document, 0, 27...|[{document, 0, 68...|[{token, 0, 1, 33...|[{token, 0, 1, 33...|[{token, 0, 1, 33...|[{token, 0, 1, 33...|[{pos, 0, 1, CD, ...|
|           [removed]|[{document, 0, 8,...|[{document, 0, 8,...|[{token, 0, 8, [r...|[{token, 0, 8, [r...|[{token, 0, 8, [r...|[{token, 0, 8, [r...|[{

In [4]:
output_path = f"{workspace_wasbs_base_url}cancer_subreddit_sentiment.parquet"
# Read the Parquet file back into a dataframe
df_read_back = spark.read.parquet(output_path)

# Show first 5 rows
df_read_back.show(5)
df_read_back.printSchema()

StatementMeta(ba5e360d-f184-47a0-9859-76b5031b79e3, 41, 9, Finished, Available, Finished)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|            sentence|               token|             checked|           sentiment|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|Check out Northsi...|[{document, 0, 26...|[{document, 0, 85...|[{token, 0, 4, Ch...|[{token, 0, 4, Ch...|[{sentiment, 0, 8...|
|I had something s...|[{document, 0, 22...|[{document, 0, 67...|[{token, 0, 0, I,...|[{token, 0, 0, I,...|[{sentiment, 0, 6...|
|That's an insulti...|[{document, 0, 22...|[{document, 0, 40...|[{token, 0, 5, Th...|[{token, 0, 5, th...|[{sentiment, 0, 4...|
|Yeah sorry, it wa...|[{document, 0, 84...|[{document, 0, 29...|[{token, 0, 3, Ye...|[{token, 0, 3, Ye...|[{sentiment, 0, 2...|
|I see my colorect...|[{document, 0, 65...|[{document, 0, 31...|[{token, 0, 0, I,...|[{token, 0, 0, I,..

In [4]:
from pyspark.sql import functions as F
df_flat = df_read_back.withColumn("sentiment_result", F.explode(F.col("sentiment"))) \
            .select("text", "sentiment_result.result")

# Show the results
df_flat.show(truncate=False)

StatementMeta(ba5e360d-f184-47a0-9859-76b5031b79e3, 39, 9, Finished, Available, Finished)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [7]:
df_flat.show(5)
comments_row_count = df_flat.count()
comment_col_count = len(df_flat.columns)
print(f"shape of the comments dataframe is {comments_row_count:,}x{comment_col_count}")

StatementMeta(ba5e360d-f184-47a0-9859-76b5031b79e3, 39, 12, Finished, Available, Finished)

+--------------------+--------+
|                text|  result|
+--------------------+--------+
|Check out Northsi...|negative|
|Check out Northsi...|negative|
|Check out Northsi...|positive|
|Check out Northsi...|negative|
|Check out Northsi...|positive|
+--------------------+--------+
only showing top 5 rows

shape of the comments dataframe is 41,883x2


In [ ]:
sentiment_df = df_read_back.selectExpr("explode(sentiment) as sentiment_struct") \
                 .select("sentiment_struct.result", "sentiment_struct.metadata")


sentiment_df.show()
